In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing.image import img_to_array
import pickle
import pandas as pd
import mediapipe as mp

# Load FER model
fer_model = model_from_json(open("Facial Detection\model.json", "r").read())
fer_model.load_weights('Facial Detection\model.h5')

# Load Haar Cascade classifier for face detection
face_cascade = cv2.CascadeClassifier('Facial Detection\haarcascade_frontalface_default.xml')
if face_cascade.empty():
    print("Error loading cascade classifier XML file.")

# Load body language model

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

with open('body_language_rf_2.pkl', 'rb') as f:
    body_language_model = pickle.load(f)

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to grayscale for face detection
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces using Haar Cascade classifier
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Iterate over each detected face
        for (x, y, w, h) in faces:
            # Extract the face ROI (Region of Interest)
            face_roi = gray_frame[y:y + h, x:x + w]

            # Resize the face ROI to match the input shape of the FER model
            resized_face = cv2.resize(face_roi, (48, 48))

            # Convert the resized face to an array
            image_pixels = img_to_array(resized_face)

            # Expand dimensions to match the model input shape
            image_pixels = np.expand_dims(image_pixels, axis=0)

            # Normalize the image
            image_pixels /= 255.0

            # Perform prediction using FER model
            predictions = fer_model.predict(image_pixels)

            # Get the predicted class
            predicted_class = np.argmax(predictions[0])

            # Map predicted class to emotion label
            emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
            emotion_label = emotion_labels[predicted_class]
            print("Emotion (FER):", emotion_label)

            # Display the emotion label on the frame
            cv2.putText(frame, emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # Draw a rectangle around the face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Process body landmarks using Mediapipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)

        # Display body landmarks
        # mp_drawing.draw_landmarks(frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
        #                            mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
        #                            mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
        #                            )

        # mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
        #                            mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
        #                            mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
        #                            )

        # mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
        #                            mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
        #                            mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
        #                            )

        # mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
        #                            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
        #                            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
        #                            )

        # Extract features from body landmarks
        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

            row = pose_row + face_row

            X = pd.DataFrame([row])
            body_language_class = body_language_model.predict(X)[0]
            body_language_prob = body_language_model.predict_proba(X)[0]
            print("Body Language:", body_language_class, body_language_prob)

            # Display body language class and probability on the frame
            cv2.putText(frame, f"Body Language: {body_language_class}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Probability: {max(body_language_prob)}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        except Exception as e:
            print("Error:", e)

        # Display the frame
        cv2.imshow('Combined Feed', frame)

        # Check if the user pressed 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the capture and close all windows
cap.release()
cv2.destroyAllWindows()


Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'lan

D:\Softwares\Anaconda\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\Softwares\Anaconda\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Body Language: neutral [0.11 0.89 0.  ]
1/1 [==============================] - 0s 53ms/step
Emotion (FER): Angry


D:\Softwares\Anaconda\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\Softwares\Anaconda\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Body Language: neutral [0.11 0.89 0.  ]
1/1 [==============================] - 0s 55ms/step
Emotion (FER): Neutral


...

Emotion (FER): Angry
1/1 [==============================] - 0s 37ms/step
Emotion (FER): Angry
Body Language: neutral [0.01 0.98 0.01]
1/1 [==============================] - 0s 34ms/step


D:\Softwares\Anaconda\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\Softwares\Anaconda\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Emotion (FER): Happy
1/1 [==============================] - 0s 37ms/step
Emotion (FER): Angry
Error: 'NoneType' object has no attribute 'landmark'


D:\Softwares\Anaconda\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\Softwares\Anaconda\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\Softwares\Anaconda\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\Softwares\Anaconda\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Body Language: neutral [0.03 0.97 0.  ]
Body Language: neutral [0.03 0.97 0.  ]
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' o